## Implementasi RNN ()

### 1. Preprocessing (Tokenization + Embedding)

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import pandas as pd

# Muat data
train_df = pd.read_csv("train.csv")

# Siapkan teks
train_texts = train_df['text'].values

# Konfigurasi tokenisasi
max_tokens = 10000
output_sequence_length = 100

# Inisialisasi layer TextVectorization
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length)
vectorizer.adapt(train_texts)

# Contoh hasil tokenisasi
sample_texts = train_df['text'].iloc[:3].values
vectorized_samples = vectorizer(sample_texts)
print(vectorized_samples.numpy())


ModuleNotFoundError: No module named 'tensorflow.python'